Process:

1. Found Twitter accounts tied to 85 airlines worldwide
1. Bookmarked their pages and scraped Twitter handles from bookmarks text file

In [1]:
with open("bookmarks_twitter_airlines.txt", "r") as myfile:
    bookmarks = myfile.read().splitlines()

In [2]:
twitter_airline_handles = [bkmk.split("(")[1].split(")")[0] for bkmk in bookmarks]

In [3]:
bearer_token = "AAAAAAAAAAAAAAAAAAAAAElqjwEAAAAA8PRQduIf5nDumMCOqlgqpkL9QHo%3D7d4vdi0UEJNMs9SmNH8nEQf9acD08THiUvzkJq3VfQzQJ4SSEf"

In [5]:
import json
with open('airline_account_handles_and_ids.json') as f:
    airline_handles_and_ids = json.load(f)

In [6]:
import requests
import os
import json
import time
from time import sleep

# To set your environment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'


def create_url(user_id):
    # Replace with user ID below
    return "https://api.twitter.com/2/users/{}/tweets".format(user_id)

def get_params(max_results, next_token=None):
    place_fields = [
        'full_name', 'id', 'country', 'country_code', 
        'name', 'place_type'
    ]
    
    tweet_fields = [
        'id', 'text', 'author_id', 'context_annotations',
        'conversation_id', 'created_at', 'entities', 
        'in_reply_to_user_id', 'lang', 'public_metrics', 
        'source', 'withheld'
    ]
    
    # user_fields = [
    #     'verified'
    # ]
    
    expansions = ['geo.place_id', 'entities.mentions.username']
    
    params = {
        "max_results": max_results,
        #'start_time': start_time,
        #'end_time': end_time,
        'exclude': 'replies',
        'expansions': ",".join(expansions),
        'tweet.fields': ",".join(tweet_fields),
        # 'user.fields': ",".join(user_fields),
        'place.fields': ",".join(place_fields)
    }
    
    # params = {"tweet.fields": "id,text,author_id,context_annotations,conversation_id,created_at", 
    #         "max_results": 5}
    
    # params = {"tweet.fields": ",".join(tweet_fields), 
    #           #'start_time': start_time,
    #           #'end_time': end_time,
    #         "max_results": 5}
    
    # if until_id != "None":
    #     params["until_id"] = until_id
    
    if next_token:
        params["pagination_token"] = next_token
        
    return params


def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2UserTweetsPython"
    return r


def connect_to_endpoint(url, params):
    response = requests.request("GET", url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()


# def main():
#     url = create_url()
#     params = get_params()
#     json_response = connect_to_endpoint(url, params)
#     print(json.dumps(json_response, indent=4, sort_keys=True))


# if __name__ == "__main__":
#     main()

In [10]:
save_dir = "airline_tweets_all_NO_REPLIES/"

start_idx = 50
end_idx = 100
for num, airline_info in enumerate(airline_handles_and_ids['data'][start_idx: end_idx]):
    
    print(start_idx + num)
    
    user_id = airline_info['id']
    user_handle = airline_info['username']
    
    print(user_handle)
    print(user_id)

    if not os.path.isdir(save_dir + user_handle):
        os.mkdir(save_dir + user_handle)

    url = create_url(user_id)

    params = get_params(max_results=100)
    json_response = connect_to_endpoint(url, params)
    sleep(5)

    path_to_save_json = save_dir + user_handle + "/" + user_handle + "_" + json_response['meta'].get('newest_id', 'last') + ".json"
    with open(path_to_save_json, 'w') as outfile:
        json.dump(json_response, outfile)
        
    print(f"\tSaved {json_response['meta'].get('result_count')} tweets to " + path_to_save_json + "...")


    next_token = json_response['meta'].get('next_token')


    while next_token:
        params = get_params(max_results=100, next_token=next_token)
        json_response = connect_to_endpoint(url, params)
        sleep(5)


        path_to_save_json = save_dir + user_handle + "/" + user_handle + "_" + json_response['meta'].get('newest_id', 'last') + ".json"
        with open(path_to_save_json, 'w') as outfile:
            json.dump(json_response, outfile)

            
        print(f"\tSaved {json_response['meta'].get('result_count')} tweets to " + path_to_save_json + "...")
        

        next_token = json_response['meta'].get('next_token')
        
print("Done.")

50
AlaskaAir
13192972
200
	Saved 100 tweets to airline_tweets_all_NO_REPLIES/AlaskaAir/AlaskaAir_1598035429160325148.json...
200
	Saved 100 tweets to airline_tweets_all_NO_REPLIES/AlaskaAir/AlaskaAir_1542867989095383040.json...
200
	Saved 100 tweets to airline_tweets_all_NO_REPLIES/AlaskaAir/AlaskaAir_1499522525306531843.json...
200
	Saved 100 tweets to airline_tweets_all_NO_REPLIES/AlaskaAir/AlaskaAir_1451671102145667074.json...
200
	Saved 100 tweets to airline_tweets_all_NO_REPLIES/AlaskaAir/AlaskaAir_1414998068047532034.json...
200
	Saved 100 tweets to airline_tweets_all_NO_REPLIES/AlaskaAir/AlaskaAir_1376651646386774021.json...
200
	Saved 100 tweets to airline_tweets_all_NO_REPLIES/AlaskaAir/AlaskaAir_1340461958395551745.json...
200
	Saved 100 tweets to airline_tweets_all_NO_REPLIES/AlaskaAir/AlaskaAir_1307458955153637376.json...
200
	Saved 50 tweets to airline_tweets_all_NO_REPLIES/AlaskaAir/AlaskaAir_1279840978220785666.json...
51
IndiGo6E
1467228402
200
	Saved 100 tweets to airl

In [9]:
json_response

{'meta': {'result_count': 0,
  'previous_token': '10iy0xk016rrqp9x9wnljt1so5tsxc33d8iyfilllzqt'}}

In [22]:
airline_info

{'id': '260187407', 'name': 'Middle East Airlines', 'username': 'MEAAIRLIBAN'}

In [13]:
print(json.dumps(json_response, indent=4, sort_keys=True))

{
    "data": [
        {
            "author_id": "2244994945",
            "context_annotations": [
                {
                    "domain": {
                        "description": "Categories within Brand Verticals that narrow down the scope of Brands",
                        "id": "46",
                        "name": "Business Taxonomy"
                    },
                    "entity": {
                        "description": "Brands, companies, advertisers and every non-person handle with the profit intent related to softwares, apps, communication equipments, hardwares",
                        "id": "1557697333571112960",
                        "name": "Technology Business"
                    }
                },
                {
                    "domain": {
                        "description": "A taxonomy of user interests. ",
                        "id": "131",
                        "name": "Unified Twitter Taxonomy"
                    },
              

In [11]:
# See how many tweets you have
import os

tweet_count = 0
save_dir = "airline_tweets_all_NO_REPLIES/"
for airline_filename in os.listdir(save_dir):
    for airline_tweet_json_filename in os.listdir(save_dir + airline_filename):
        try:
            with open(f'{save_dir}{airline_filename}/{airline_tweet_json_filename}') as f:
                data = json.load(f)
            tweet_count += data['meta']['result_count']
            print(tweet_count)
        except IsADirectoryError:
            continue

100
150
250
350
450
550
650
750
850
950
1000
1100
1199
1298
1398
1498
1598
1698
1798
1897
1996
2095
2195
2295
2395
2445
2545
2595
2695
2795
2895
2995
3095
3195
3295
3395
3495
3595
3645
3745
3845
3945
4045
4145
4245
4345
4445
4545
4636
4735
4835
4835
4935
5035
5135
5185
5285
5385
5485
5585
5685
5785
5885
5984
6084
6183
6282
6382
6479
6579
6579
6675
6775
6875
6975
7075
7102
7202
7302
7402
7502
7602
7702
7802
7902
7909
8007
8107
8207
8304
8404
8504
8604
8704
8804
8904
9004
9015
9115
9215
9315
9415
9465
9565
9665
9765
9865
9965
10065
10165
10265
10267
10367
10467
10567
10667
10767
10867
10967
11067
11167
11267
11366
11466
11563
11575
11675
11775
11875
11975
12075
12175
12275
12275
12375
12473
12573
12673
12773
12873
12973
12979
13079
13179
13279
13379
13479
13529
13629
13729
13829
13929
14029
14129
14229
14329
14429
14529
14629
14678
14778
14878
14978
15078
15178
15278
15328
15428
15528
15628
15657
15757
15857
15957
16057
16157
16257
16357
16456
16555
16555
16652
16750
16849
16948
17048
17